In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import scipy.interpolate
import os
import matplotlib.pyplot as plt
import scipy.interpolate
import matplotlib.gridspec as gridspec
import ctypes
from fatiando.gravmag import transform
import os.path
import matplotlib.image as mpimg

def findPeak(data,alpha): #Open .txt mag data and plot the power spectrum related to certain alpha
    
    #open Mag data
    x,y,mag = np.loadtxt(data, usecols=(0,1,2), unpack = True, delimiter=',') 
    
    #Prepare data to calculate power density spectrum   
    N = int(np.sqrt(len(mag)))       
    shape = (N,N)
    x_grid = np.linspace(x.min(), x.max(), N)
    y_grid = np.linspace(y.min(), y.max(), N)
    xi,yi = np.meshgrid(x_grid,y_grid)
    zi = scipy.interpolate.griddata((x, y), mag,
                                    (xi,yi), method='cubic')
    
    #Calculate 2D power spectrum and radial average
    kx, ky, pds = transform.power_density_spectra(xi, yi, zi, shape) 
    k_radial = np.sqrt(kx*kx + ky*ky)
    pds_defractal = pds*(k_radial**alpha) #Defract the observed power density spectrum
    k_radial, pds_radial = transform.radial_average_spectrum(kx, ky, pds_defractal)
    k_radial = k_radial*1000 #Adjust unit from rad/m to rad/km
    
    #Plot radial average
    plt.scatter(k_radial,pds_radial,label='alpha='+str(alpha))
    
#Define the point position k Range for Zt and Z0
Kzt_min = 5
Kzt_max = 7
Kz0_min = 0
Kz0_max = 3

def defractal(data,Kzt_min,Kzt_max,Kz0_min,Kz0_max):
    #open Mag data
    x,y,mag = np.loadtxt(data, usecols=(0,1,2), unpack = True, delimiter=',') 
    
    #Prepare data to calculate power density spectrum   
    N = int(np.sqrt(len(mag)))       
    shape = (N,N)
    x_grid = np.linspace(x.min(), x.max(), N)
    y_grid = np.linspace(y.min(), y.max(), N)
    xi,yi = np.meshgrid(x_grid,y_grid)
    zi = scipy.interpolate.griddata((x, y), mag,
                                    (xi,yi), method='cubic')
    
    #Calculate 2D power spectrum and radial average
    kx, ky, pds = transform.power_density_spectra(xi, yi, zi, shape) 
    k_radial = np.sqrt(kx*kx + ky*ky)
    alpha = 0
    while alpha <= 4:
        pds_defractal = pds*(k_radial**alpha) #Defract the observed power density spectrum
        k_radial, pds_radial = transform.radial_average_spectrum(kx, ky, pds_defractal)
        k_radial = k_radial*1000 #Adjust unit from rad/m to rad/km
        
        #Fit line using K ranges for Zt
        m, b1 = np.polyfit(k_radial[Kzt_min:Kzt_max], np.log(pds_radial)[Kzt_min:Kzt_max], 1)
        zt = abs(m)/2
        
        #Fit line using K ranges for Z0
        m, b2 = np.polyfit(k_radial[Kz0_min:Kz0_max], np.log(np.sqrt(pds_radial)/k_radial)[Kz0_min:Kz0_max], 1)
        z0 = abs(m)
        zb = 2*z0 - zt
        
        #calculate RMS between calculated and modelled
        model = (np.log(np.exp(b1)*((1/np.exp(abs(k_radial)*zt)) - 1/np.exp(abs(k_radial)*zb))**2))
        soma = np.sum((model[1:]-pds_radial[1:])**2)
        rms = np.sqrt(soma/len(model))
        
        #save figure with ln(power) centroid, ln(power) calculated, ln(power) modelled
        
        #save txt with x,y,zt,zb,alpha, rms
        
        alpha =+0.2
        